# Linear Regression

In [120]:
# Libraries
import pandas as pd
import numpy as np
import bokeh
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import NumeralTickFormatter
output_notebook()
from bokeh.layouts import row
from pandas.api.types import is_numeric_dtype

Loading BokehJS ...

In [121]:
# Load data
dataset = pd.read_csv("../dataset/student-mat.csv", delimiter=";")

In [122]:
# Choose features and target
target = "G3"
skiped_features = ["G1", "G2"]

features = list(dataset.columns)
features.remove(target)
features = np.setdiff1d(features, skiped_features)
print(features)

['Dalc' 'Fedu' 'Fjob' 'Medu' 'Mjob' 'Pstatus' 'Walc' 'absences'
 'activities' 'address' 'age' 'failures' 'famrel' 'famsize' 'famsup'
 'freetime' 'goout' 'guardian' 'health' 'higher' 'internet' 'nursery'
 'paid' 'reason' 'romantic' 'school' 'schoolsup' 'sex' 'studytime'
 'traveltime']


In [123]:
# Convert nominal features and/or target to numeric values

# This function get a dataset as parameter and for every nominal feature makes
# a dictionary which maps every original nominal value of the feature to the 
# new numeric value. Original numeric features are skiped. The function returns
# dictionary where keys are nominal features from the input dataset and values are nested dictionares
# of feature values mappings as described above.
def nom_to_num(dataset):
    raw_dictionary = []
    for feature in dataset.columns:
        if not is_numeric_dtype(type(dataset.loc[0,feature])):
            nominals = np.unique(dataset.loc[:,feature])
            enumeration = np.arange(len(nominals))
            mapping = dict(list(zip(nominals, enumeration)))
            raw_dictionary.append((feature, mapping))
    return dict(raw_dictionary)

# Below function converts given dataset to a dataset with only numerical values
def to_numeric(dataset):
    dataset = dataset.copy()
    mapping = nom_to_num(dataset)
    features = list(mapping.keys())
    dataset[features] = dataset[features].apply(lambda col: col.map(lambda val: mapping[col.name][val]))
    return dataset

dataset = to_numeric(dataset)

In [124]:
def standardize(dataset):
    dataset = dataset.copy()
    dataset = (dataset - dataset.apply(np.mean))/(dataset.apply(np.max) - dataset.apply(np.min))
    return dataset

dataset = standardize(dataset)

# display(dataset)

In [125]:
# # Add square features
def add_square_features(X):
    X2 = X**2
    X2.columns = X2.columns.map(lambda label: label + "2")
    X2 = pd.concat([X, X2], axis=1)
    new_features = X2.columns.values
    return X2, new_features


In [126]:
# Prepare datasets
X = dataset.loc[:,features]
X, features = add_square_features(X)
X=X.values
Y = dataset.loc[:,target].values

dataset_length = len(X)
cv_start_index = int(0.6*dataset_length)
test_start_index = int(0.8*dataset_length)

X_training = X[:cv_start_index]
Y_training = Y[:cv_start_index]
X_cv = X[cv_start_index:test_start_index]
Y_cv = Y[cv_start_index:test_start_index]
X_test = X[test_start_index:]
Y_test = Y[test_start_index:]

In [127]:
features

array(['Dalc', 'Fedu', 'Fjob', 'Medu', 'Mjob', 'Pstatus', 'Walc',
       'absences', 'activities', 'address', 'age', 'failures', 'famrel',
       'famsize', 'famsup', 'freetime', 'goout', 'guardian', 'health',
       'higher', 'internet', 'nursery', 'paid', 'reason', 'romantic',
       'school', 'schoolsup', 'sex', 'studytime', 'traveltime', 'Dalc2',
       'Fedu2', 'Fjob2', 'Medu2', 'Mjob2', 'Pstatus2', 'Walc2',
       'absences2', 'activities2', 'address2', 'age2', 'failures2',
       'famrel2', 'famsize2', 'famsup2', 'freetime2', 'goout2',
       'guardian2', 'health2', 'higher2', 'internet2', 'nursery2',
       'paid2', 'reason2', 'romantic2', 'school2', 'schoolsup2', 'sex2',
       'studytime2', 'traveltime2'], dtype=object)

In [128]:
# Hypothesis function
def h(THETA, X):
    return X.dot(THETA)

In [129]:
# Cost function
def J(THETA, X, Y, LAMBDA):
    return 1/(2*len(X)) * (np.sum((h(THETA, X)-Y)**2) + LAMBDA * np.sum(THETA**2))

# Derrivative of cost function respects to i-th THETA parameter
def dJ(THETA, X, Y, LAMBDA, i):
    return 1/(len(X)) * (np.sum((h(THETA, X)-Y)*X[:,i]) + LAMBDA * THETA[i])

In [130]:
# Gradient Descent
def gradient_descent(X, Y, a, LAMBDA):
    costs = []
    THETA = np.zeros(X.shape[1])
    while True:
        NEW_THETA = np.copy(THETA)
        for i in range(len(THETA)):
            NEW_THETA[i] = THETA[i] - a*dJ(THETA, X, Y, LAMBDA, i)
        costs.append(J(THETA, X, Y, LAMBDA))
        if J(NEW_THETA, X, Y, LAMBDA) <= J(THETA, X, Y, LAMBDA):
            THETA = NEW_THETA
        else:
            return THETA, costs

In [131]:
%%time
a = 0.1 # Learning rate
LAMBDA = 10 # Regularization parameter
THETA, costs = gradient_descent(X_training, Y_training, a, LAMBDA)
print(THETA)

[-0.0266258  -0.00110634  0.05510124  0.05709092  0.02969942 -0.01487455
  0.00420097 -0.00704583  0.03841093  0.028211   -0.04889941 -0.14112261
  0.02873291  0.0218028  -0.04770149  0.03096237 -0.09437088 -0.0066656
 -0.04079276  0.03824964  0.01725726  0.00598072  0.02361746  0.02224291
 -0.05070914 -0.00127771 -0.04120957  0.03575543  0.00642731 -0.04730221
  0.00713928  0.01321462  0.00829991  0.02392159 -0.03724348  0.01280728
  0.0225859  -0.00746475  0.00206134 -0.01374128  0.00218049 -0.06749348
  0.00513091  0.01147051  0.01335158  0.02051386  0.01031211 -0.000367
  0.02175354 -0.03384886 -0.00996336 -0.00173936  0.00469686 -0.01242619
 -0.01437625  0.0001488  -0.02933443  0.00463607  0.00901151  0.00919442]
CPU times: user 11.7 s, sys: 4.35 s, total: 16.1 s
Wall time: 8.28 s


In [132]:
p = figure(title = "Cost function value against iterations",
           x_axis_type="log", x_axis_label="Iterations",
           y_axis_label="Cost function value",
          width=500, height=500)
iterations = list(range(0,len(costs)))
p.line(iterations, costs)
p.line(iterations, costs[-1], line_color="red")

def accuracy(THETA, X, Y, LAMBDA):
    J_MAX = 2.0
    return J(THETA, X, Y, LAMBDA)/J_MAX

from bokeh.models import LinearAxis, Range1d
    
J(THETA, X_cv, Y_cv, LAMBDA)
J(THETA, X_test, Y_test, LAMBDA)

accuracies = np.array(costs)/2.0
p1 = figure(title = "Cost function percentage against iterations",
           x_axis_type="log", x_axis_label="Iterations",
           y_axis_label="Cost function percentage",
          width=500, height=500)
iterations = list(range(0,len(costs)))
p1.yaxis.formatter = NumeralTickFormatter(format='0.f %')
p1.line(iterations, accuracies)
p1.line(iterations, accuracies[-1], line_color="red")

show(row(p,p1))

In [153]:
# Importance of the features

# Group features
def group_features(theta):
    theta = theta.copy()
    square_features = []
    for col in theta.columns:
        if "2" in col:
            square_features.append(col)
            theta.loc[:,col[:-1]]+=theta.loc[:,col]
    return theta.drop(square_features, axis=1)



# Normalize
# absolute_theta = np.absolute(THETA)
# normalized_theta = absolute_theta/np.sum(absolute_theta)

DF_THETA = pd.DataFrame(data=[THETA], columns=features)

# Normalization
absolute_theta = DF_THETA.abs()
features_importance = group_features(absolute_theta) # Merge polynomial features (x = x + x^2)
normalized_features_importance = features_importance.divide(features_importance.loc[0,:].sum())

# Sort importance
sorted_features_importance = features_importance.sort_values(by=0,axis=1)
sorted_importance = sorted_features_importance.loc[0].values
sorted_features = sorted_features_importance.columns.values

from bokeh.models import NumeralTickFormatter


p1 = figure(title = "Importance of features",
            x_axis_label="Features", y_axis_label="Importance",
            x_range=sorted_features,
          width=800, height=500)
p1.yaxis.formatter = NumeralTickFormatter(format='0 %')
p1.xaxis.major_label_orientation = "vertical"
p1.vbar(x=sorted_features, top=sorted_importance, width=1,
       fill_color='orange', line_color='black')
show(p1)

# Weights of the features
features_weights = pd.DataFrame(data=[THETA], columns=features)
sorted_features_weights = features_weights.sort_values(by=0,axis=1)
sorted_weights = sorted_features_weights.loc[0].values
sorted_features = sorted_features_weights.columns.values

p2 = figure(title = "Features weights",
            x_axis_label="Features", y_axis_label="Weights",
            x_range=sorted_features,
          width=800, height=500)
p2.xaxis.major_label_orientation = "vertical"
p2.vbar(x=sorted_features, top=sorted_weights, width=1,
       fill_color='blue', line_color='black')
show(p2)